In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from Dataset import BitcoinData
from TradingEnvironment import TradingEnvironment


tf.keras.utils.disable_interactive_logging()


In [ ]:
bitcoin_data = BitcoinData()
from Models.DuelingDDQN import DuelingDoubleDQN

def trainDDDQN():
    environment = TradingEnvironment(bitcoin_data)

    observation_space = environment.get_observation_space()
    action_space = environment.get_action_space()
    episode = 0

    agentoo7 = DuelingDoubleDQN(observation_space=observation_space, action_space=action_space, model_path="DDDQN",
                         load_version=-1, load_model_num=-1)   

    NUM_EPISODES = 20
    for episode in range(NUM_EPISODES):
        agentoo7.model_num_save = episode
        done = False
        state = environment.reset()
        total_reward = 0
        step = 0


        while not done:
            step+=1
            action = agentoo7.act(state)
            next_state, reward, done, _ = environment.step(action - 1)
            agentoo7.update_mem(state, action, reward, next_state, done)
            agentoo7.train()
            state = next_state
            total_reward += reward
            
            if (step - 1) % 100 == 0:
                print("Step {}:".format(step - 1))
                print("    Percent Capital: {:.4f}, interval: [{:.4f}, {:.4f}]".format(
                    environment.percent_profit, environment.min_percent_profit, environment.max_percent_profit
                ))
                print("    Action taked by itseft: SELL: {}, HOLD: {}, BUY: {}".format(
                    agentoo7.actions_taked_by_itself[0], agentoo7.actions_taked_by_itself[1], agentoo7.actions_taked_by_itself[2]
                ))

                agentoo7.actions_taked_by_itself = [0, 0, 0]

            if done:
                print("Episode {}/{} ended\n".format(episode, NUM_EPISODES))
                break
            
        agentoo7.save_model()
              
    return agentoo7
        
agent = trainDDDQN()


In [ ]:
last_episode = 40
start_time = TradingEnvironment.NUM_OBSERVATIONS * last_episode
load_version = 9
load_model_num = 20

from Models.DuelingDDQN import DuelingDoubleDQN


def get_environment(start_time=0):
    bitcoin_data = BitcoinData()

    return TradingEnvironment(bitcoin_data, start_time=start_time)

def get_agent(environment, load_version=-1, load_model_num=-1):
    observation_space = environment.get_observation_space()
    action_space = environment.get_action_space()

    
    return DuelingDoubleDQN(observation_space=observation_space, action_space=action_space, model_path="DuellingDDQN",
                         load_version=load_version, load_model_num=load_model_num)   


environment = get_environment(start_time=start_time)
# agentoo7 = agent
agentoo7 = get_agent(environment=environment, load_version=load_version, load_model_num=load_model_num)
agentoo7.epsilon = 0.0

NUM_EPISODES = 10

for episode in range(NUM_EPISODES):
        done = False
        state = environment.reset()
        total_reward = 0
        step = 0

        while not done:
            step+=1
            action = agentoo7.act(state)
            next_state, reward, done, _ = environment.step(action - 1)
            state = next_state
            total_reward += reward

            if (step - 1) % 100 == 0:
                print("Step {}:".format(step - 1))
                print("    Percent Capital: {:.4f}, interval: [{:.4f}, {:.4f}]".format(
                    environment.percent_profit, environment.min_percent_profit, environment.max_percent_profit
                ))
                print("    Action taked by itseft: SELL: {}, HOLD: {}, BUY: {}".format(
                    agentoo7.actions_taked_by_itself[0], agentoo7.actions_taked_by_itself[1], agentoo7.actions_taked_by_itself[2]
                ))

                agentoo7.actions_taked_by_itself = [0, 0, 0]

            if done:
                print("Episode {}/{} ended\n".format(episode, NUM_EPISODES))
                break

In [13]:
f = open("input_txt.txt", 'r').read()
import re

def filter_episodes(text, num_ep):
    pattern = re.compile(r"Step 8400:\s+Percent Capital: ([\d.]+), interval: \[([\d.]+), ([\d.]+)\]\s")
    matches = pattern.findall(text)

    result = []
    i = 1

    for match in matches:
        if i < num_ep:
            percent_capital, interval_start, interval_end = match
            result.append(f"Episode {i}/{num_ep} ended: Percent Capital: {percent_capital}, interval: [{interval_start}, {interval_end}]")
            i+=1
        else:
            break

    return result

result = filter_episodes(f, 10)
mean = []

for episode_info in result:
    print(episode_info)
    # print(episode_info.split(', ')[-1][:-1])
    # print(episode_info.split(', ')[-2].split('[')[1])
    # print(x)
    mean.append(float(episode_info.split(', ')[-1][:-1]))
    

print("Media episoadelor: ", sum(mean)/len(mean))

Episode 1/10 ended: Percent Capital: 1.0915, interval: [0.9919, 1.1007]
Episode 2/10 ended: Percent Capital: 1.0769, interval: [0.9906, 1.1013]
Episode 3/10 ended: Percent Capital: 1.1329, interval: [0.9945, 1.1383]
Episode 4/10 ended: Percent Capital: 1.3621, interval: [0.9716, 1.4147]
Episode 5/10 ended: Percent Capital: 0.9967, interval: [0.9719, 1.0510]
Episode 6/10 ended: Percent Capital: 1.0782, interval: [0.9909, 1.1127]
Episode 7/10 ended: Percent Capital: 1.0069, interval: [0.9708, 1.0352]
Episode 8/10 ended: Percent Capital: 1.1841, interval: [0.9819, 1.2054]
Episode 9/10 ended: Percent Capital: 0.9712, interval: [0.9442, 1.0351]
Media episoadelor:  1.132711111111111
